In [22]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm
import json
import re
import shap
import lightgbm as lgb
import itertools
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score

from utils.utils import *
from utils.pre_processing import *
from utils.nn import *
from utils.nn_utils import *

In [39]:
df = load_data('data/df_tags','data/final_reviews')
X_train, X_test, X_val, y_val, y_train, y_test = init_df(df,test_size=0.2,target_col='target',stratify=True,scaling=True,oversampling=True)
X_train, X_test, X_val, to_remove = remove_highly_correlated_features(X_train, X_test, X_val, threshold=0.9)
df = df.drop(columns=df.columns[to_remove])

In [40]:
feature_names = df.columns[:-1]  # Assuming the last column is the target

# Create LightGBM datasets
lgb_train = lgb.Dataset(X_train, label=y_train, feature_name=list(feature_names))
lgb_val = lgb.Dataset(X_val, label=y_val, feature_name=list(feature_names), reference=lgb_train)

# Define the model parameters
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 31,
    'learning_rate': 0.01,
    'feature_fraction': 1
}

def f1_eval(y_pred, dataset):
    y_true = dataset.get_label()
    y_pred_binary = np.round(y_pred)
    return 'f1', f1_score(y_true, y_pred_binary), True

# Train the model
model = lgb.train(
    params, 
    lgb_train, 
    num_boost_round=100, 
    valid_sets=[lgb_train, lgb_val], 
    valid_names=['train', 'val'],
    feval=f1_eval
)

# Make predictions on the validation set
y_val_pred = model.predict(X_val, num_iteration=model.best_iteration)
y_val_pred_binary = np.round(y_val_pred)

# Calculate evaluation metrics
accuracy = accuracy_score(y_val, y_val_pred_binary)
precision = precision_score(y_val, y_val_pred_binary)
recall = recall_score(y_val, y_val_pred_binary)
f1 = f1_score(y_val, y_val_pred_binary)

# Print the evaluation metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# Get feature importance
importance = model.feature_importance()
feature_importance = pd.DataFrame({'Feature': feature_names, 'Importance': importance})

# Sort by importance
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

# Return the feature importance and metrics
evaluation_metrics = {
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1_score': f1
}

quantile_to_keep = 0.5
min_imp = feature_importance['Importance'].quantile(quantile_to_keep)
imp_feats = feature_importance.loc[feature_importance['Importance'] >= min_imp]['Feature']

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2705, number of negative: 2705
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006699 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 75579
[LightGBM] [Info] Number of data points in the train set: 5410, number of used features: 304
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Accuracy: 0.8263305322128851
Precision: 0.1
Recall: 0.17307692307692307
F1 Score: 0.1267605633802817


In [54]:
feature_importance.sort_values('Importance')['Feature'].head(20)

254                        Werewolves
290                      Outbreak Sim
289    Intentionally Awkward Controls
56                     Hack and Slash
287                       Programming
286                     Warhammer 40K
57                             Nudity
278                             Party
293                         Benchmark
283                       Time Attack
282                       Agriculture
280                 Time Manipulation
63                   Great Soundtrack
294                     Political Sim
82                       Early Access
205                        Minimalist
258                      Auto Battler
259          Asynchronous Multiplayer
35                             Aliens
285                             Naval
Name: Feature, dtype: object

In [23]:
df = load_data('data/df_tags','data/final_reviews')

X_train, X_test, y_train, y_test = init_df(df,test_size=0.2,target_col='target',stratify=True,scaling=False,oversampling=False,val=False)
X_train = X_train.values
X_test = X_test.values

X_train_reduced = np.delete(X_train, to_remove, axis=1)
X_test_reduced = np.delete(X_test, to_remove, axis=1)

reduced_X_train = X_train[:, imp_feats.index]
reduced_X_test = X_test[:, imp_feats.index]

In [55]:
reduced_X_train.shape

(3569, 166)

In [57]:
#constantes

input_size = reduced_X_train.shape[1]
output_size = 1
num_epochs = 1000
hidden_activation = 'relu'
output_activation='sigmoid'
criterion = 'BCEWithLogitsLoss'
device = 'cpu'

In [84]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import pandas as pd

K = 1

# Define your hyperparameters
hidden_layers = [1, 2]
hidden_layer_max_sizes = [256, 128, 64, 32, 16]
lrs = [0.01, 0.001, 0.0001, 0.00001]
patience = [10]

if K > 1:
    k_fold = KFold(n_splits=K)  # Specify K as the number of folds

results = []

for h_layers, h_size, lr, pat in itertools.product(hidden_layers, hidden_layer_max_sizes, lrs, patience):

    hidden_layers_struct = get_layers(h_layers,h_size)
    
    fold_metrics = {'Accuracy': [], 'Precision': [], 'Recall': [], 'F1': []}

    if K == 1:
        dim = int(reduced_X_train.shape[0])
        dim_thres = int(dim*0.5)
        fold_generator = [(np.arange(0,dim_thres),np.arange(dim_thres,dim))]
    else:
        fold_generator = k_fold.split(reduced_X_train)
        
    for train_index, val_index in fold_generator:
        # Split data into train and validation set for this fold
        X_train_fold, X_val_fold = reduced_X_train[train_index], reduced_X_train[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

        scaler = StandardScaler()
        X_train_fold = scaler.fit_transform(X_train_fold)
        X_val_fold = scaler.transform(X_val_fold)
        
        smote = SMOTE(random_state=42)
        X_train_fold, y_train_fold = smote.fit_resample(X_train_fold, y_train_fold)

        # Create datasets and model
        train_loader, _, val_dataset = create_datasets(X_train_fold, y_train_fold, X_test, y_test, X_val_fold, y_val_fold, val=True, batch_size=320)
        model = NeuralNetwork(input_size, hidden_layers_struct, output_size, lr, device=device, hidden_activation=hidden_activation, output_activation=output_activation, criterion=criterion)

        # Train the model
        train_nn(model, train_loader, val_dataset, num_epochs=num_epochs, skip=50, patience=pat)

        # Evaluate the model on validation set for this fold
        y_pred = predict_nn(model, val_dataset)
        accuracy, precision, recall, f1 = eval_predict_nn(y_val_fold, y_pred)

        # Store metrics for this fold
        fold_metrics['Accuracy'].append(accuracy)
        fold_metrics['Precision'].append(precision)
        fold_metrics['Recall'].append(recall)
        fold_metrics['F1'].append(f1)

    # Calculate mean metrics across all folds
    mean_accuracy = np.mean(fold_metrics['Accuracy'])
    mean_precision = np.mean(fold_metrics['Precision'])
    mean_recall = np.mean(fold_metrics['Recall'])
    mean_f1 = np.mean(fold_metrics['F1'])

    # Store hyperparameters and mean metrics in results list
    results.append({
        'Hidden Layers': h_layers,
        'Hidden Layer Size': h_size,
        'Learning Rate': lr,
        'Patience': pat,
        'Mean Accuracy': mean_accuracy,
        'Mean Precision': mean_precision,
        'Mean Recall': mean_recall,
        'Mean F1': mean_f1
    })

# Convert results to a DataFrame
results_df = pd.DataFrame(results)
results_df.to_csv('new_model_results_no_CV.csv')

  0%|          | 0/1000 [00:00<?, ?it/s]

Early stopping at epoch 12
Accuracy: 0.5647058823529412
Precision: 0.2857142857142857
Recall: 0.002583979328165375
F1: 0.005121638924455826


  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 050: | Train Loss: 0.566 | Val Loss: 0.7032 | Train F1: 0.8395| Val F1: 0.0535
Epoch 100: | Train Loss: 0.543 | Val Loss: 0.6983 | Train F1: 0.8881| Val F1: 0.0564
Epoch 150: | Train Loss: 0.535 | Val Loss: 0.6965 | Train F1: 0.8922| Val F1: 0.0494
Early stopping at epoch 173
Accuracy: 0.569187675070028
Precision: 0.5675675675675675
Recall: 0.027131782945736434
F1: 0.051787916152897656


  0%|          | 0/1000 [00:00<?, ?it/s]

...

Epoch 050: | Train Loss: 0.619 | Val Loss: 0.7054 | Train F1: 0.5603| Val F1: 0.0508
Epoch 100: | Train Loss: 0.579 | Val Loss: 0.7008 | Train F1: 0.7254| Val F1: 0.0760
Epoch 150: | Train Loss: 0.559 | Val Loss: 0.6992 | Train F1: 0.8091| Val F1: 0.0948
Early stopping at epoch 175
Accuracy: 0.5708683473389355
Precision: 0.5625
Recall: 0.046511627906976744
F1: 0.08591885441527446


  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 050: | Train Loss: 0.719 | Val Loss: 0.7684 | Train F1: 0.5528| Val F1: 0.6049
Epoch 100: | Train Loss: 0.712 | Val Loss: 0.7611 | Train F1: 0.5003| Val F1: 0.5728
Epoch 150: | Train Loss: 0.702 | Val Loss: 0.7519 | Train F1: 0.4426| Val F1: 0.3463
Epoch 200: | Train Loss: 0.692 | Val Loss: 0.7425 | Train F1: 0.4245| Val F1: 0.2100
Epoch 250: | Train Loss: 0.681 | Val Loss: 0.7342 | Train F1: 0.4130| Val F1: 0.1564
Epoch 300: | Train Loss: 0.670 | Val Loss: 0.7284 | Train F1: 0.4195| Val F1: 0.1384
Epoch 350: | Train Loss: 0.661 | Val Loss: 0.7246 | Train F1: 0.4382| Val F1: 0.1326
Epoch 400: | Train Loss: 0.653 | Val Loss: 0.7223 | Train F1: 0.4854| Val F1: 0.1351
Epoch 450: | Train Loss: 0.646 | Val Loss: 0.7210 | Train F1: 0.4751| Val F1: 0.1522
Epoch 500: | Train Loss: 0.638 | Val Loss: 0.7196 | Train F1: 0.5201| Val F1: 0.1660
Epoch 550: | Train Loss: 0.631 | Val Loss: 0.7186 | Train F1: 0.5423| Val F1: 0.1763
Epoch 600: | Train Loss: 0.624 | Val Loss: 0.7175 | Train F1: 0.5

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 050: | Train Loss: 0.712 | Val Loss: 0.7518 | Train F1: 0.0377| Val F1: 0.0076
Epoch 100: | Train Loss: 0.711 | Val Loss: 0.7515 | Train F1: 0.0404| Val F1: 0.0026
Epoch 150: | Train Loss: 0.711 | Val Loss: 0.7511 | Train F1: 0.0307| Val F1: 0.0000
Epoch 200: | Train Loss: 0.711 | Val Loss: 0.7508 | Train F1: 0.0661| Val F1: 0.0000
Epoch 250: | Train Loss: 0.710 | Val Loss: 0.7506 | Train F1: 0.0547| Val F1: 0.0000
Epoch 300: | Train Loss: 0.710 | Val Loss: 0.7503 | Train F1: 0.0684| Val F1: 0.0000
Epoch 350: | Train Loss: 0.710 | Val Loss: 0.7501 | Train F1: 0.0406| Val F1: 0.0000
Epoch 400: | Train Loss: 0.709 | Val Loss: 0.7498 | Train F1: 0.0699| Val F1: 0.0000
Epoch 450: | Train Loss: 0.709 | Val Loss: 0.7495 | Train F1: 0.0760| Val F1: 0.0000
Epoch 500: | Train Loss: 0.709 | Val Loss: 0.7492 | Train F1: 0.0891| Val F1: 0.0000
Epoch 550: | Train Loss: 0.708 | Val Loss: 0.7489 | Train F1: 0.0947| Val F1: 0.0000
Epoch 600: | Train Loss: 0.708 | Val Loss: 0.7486 | Train F1: 0.0

In [30]:
results_df

,Hidden Layers,Hidden Layer Size,Learning Rate,Patience,Mean Accuracy,Mean Precision,Mean Recall,Mean F1
0,1,256,0.0100,10,0.921825,0.115083,0.061747,0.078909
1,1,256,0.0010,10,0.923785,0.157987,0.081296,0.105608
2,1,256,0.0001,10,0.885690,0.108361,0.136685,0.110159
3,1,256,0.0001,10,0.875303,0.097406,0.130779,0.104775
4,1,128,0.0100,10,0.915101,0.093136,0.071778,0.078036
5,1,128,0.0010,10,0.916782,0.111228,0.076174,0.087632
6,1,128,0.0001,10,0.879222,0.119208,0.151701,0.126239
7,1,128,0.0001,10,0.868579,0.088567,0.135901,0.105568
8,1,64,0.0100,10,0.909777,0.085714,0.077041,0.077174
9,1,64,0.0010,10,0.897732,0.102269,0.098971,0.092541


In [7]:
results_df = pd.DataFrame(results)
results_df

,Hidden Layers,Hidden Layer Size,Learning Rate,Patience,Mean Accuracy,Mean Precision,Mean Recall,Mean F1
0,1,256,0.10,100,0.500000,0.000000,0.000000,0.000000
1,1,256,0.01,100,0.790388,0.650217,0.842482,0.639973
